<a href="https://colab.research.google.com/github/mallikharjun07/ML/blob/master/SMS%20Spam%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Clone the entire repo.
!git clone -l -s git://github.com/mallikharjun07/ML cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 15), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (27/27), 2.03 MiB | 4.72 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/content/cloned-repo
 bbc_news_mixed.csv   README.md  'SMS Spam Classification.ipynb'   spamdata.csv


In [0]:
file = '/content/cloned-repo/spamdata.csv'

In [13]:
import pandas as pd
data = pd.read_csv(file)
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
from sklearn.preprocessing import LabelEncoder
lencod = LabelEncoder()
target = lencod.fit_transform(data['label'])

In [15]:
target[:5]

array([0, 0, 1, 0, 0])

#### Load keras libraries

In [0]:

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras import models, optimizers
from keras.layers import Input,Embedding,SpatialDropout1D,Conv1D,GlobalMaxPool1D,Dense
from keras.layers import Dropout
from keras.models import Model

#### Tokenize the data

In [0]:
token = Tokenizer()
token.fit_on_texts(data["text"])
word_index = token.word_index
# type(word_index) - it is dictionary

#### split the data into training and validation datasets

In [0]:
trainx, valx, trainy, valy = train_test_split(data["text"], target)

train_seq_x = pad_sequences(token.texts_to_sequences(trainx),maxlen=70)
val_seq_x = pad_sequences(token.texts_to_sequences(valx),maxlen=70)

In [25]:
train_seq_x[:100]

array([[   0,    0,    0, ...,    3,  224,   12],
       [   0,    0,    0, ...,   47, 8857, 2985],
       [   0,    0,    0, ...,   17,    2,  414],
       ...,
       [   0,    0,    0, ...,  311,   29,  283],
       [   0,    0,    0, ..., 3209,    7,  536],
       [   0,    0,    0, ...,   11, 1315, 1413]], dtype=int32)

### Build the network architecture

In [0]:
def create_cnn():
    input_layer = Input((70,))
    embedding_layer = Embedding(len(word_index)+1,300)(input_layer)
    embedding_layer = SpatialDropout1D(0.3)(embedding_layer)

    # add the convolutional layer
    conv_layer = Conv1D(100,3,activation='relu')(embedding_layer)

    # Add the pooling layer
    pooling_layer = GlobalMaxPool1D()(conv_layer)
    output_layer1 = Dense(50,activation='relu')(pooling_layer)
    output_layer1 = Dropout(0.25)(output_layer1)
    output_layer2 = Dense(1,activation='sigmoid')(output_layer1)


    # compile the model
    model = Model(inputs = input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(),loss='binary_crossentropy')
    
    return model

In [0]:
model = create_cnn()

### Model training

In [33]:
model.fit(train_seq_x,trainy)




Epoch 1/1





4179/4179 [==============================] - 13s 3ms/step - loss: 0.2951


### Model evaluation

In [65]:
predictions = model.predict(val_seq_x)
predictions

array([[9.0700996e-01],
       [1.0342896e-03],
       [1.4178246e-02],
       ...,
       [1.5323758e-03],
       [9.6052760e-01],
       [7.3464989e-04]], dtype=float32)

In [63]:
predictions = predictions.argmax(axis=-1)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [64]:
accuracy_score(predictions,valy)

0.8628858578607322